In [3]:
from mpa.models import Mpa, mpa_post_save
from django.db.models.signals import post_save, post_delete
from mpa.views import mpas_all_nogeom
import reversion
from reversion.models import Revision
from reversion.models import Version
import datetime
from django.db.models import Q, F, Func

In [81]:
mpaset = mpas_all_nogeom.filter(
    Q(country__icontains='FRA') | Q(sovereign__icontains='FRA')
    | Q(country__icontains='ATF') | Q(country__icontains='BLM') | Q(country__icontains='GLP')
    | Q(country__icontains='GUF') | Q(country__icontains='MAF') | Q(country__icontains='MTQ')
    | Q(country__icontains='MYT') | Q(country__icontains='NCL') | Q(country__icontains='PYF')
    | Q(country__icontains='REU') | Q(country__icontains='SHN') | Q(country__icontains='SYC')
)
mpaset.values('country').distinct()

<QuerySet [{'country': 'MAF'}, {'country': 'GUF'}, {'country': 'ATF'}, {'country': 'MYT'}, {'country': 'MTQ'}, {'country': 'GLP'}, {'country': 'SYC'}, {'country': 'BLM'}, {'country': 'PYF'}, {'country': 'FRA'}, {'country': 'SHN'}, {'country': 'NCL'}, {'country': 'REU'}, {'country': 'ESP/FRA'}]>

In [70]:
mpas_all_nogeom.filter(wdpa_id=0).update(wdpa_id=None)
mpas_all_nogeom.filter(wdpa_id=0).count()

0

In [71]:
count = 0
for m in mpaset:
    versions = Version.objects.get_for_object(m)
    created = 'Never'
    if not m.wdpa_id: continue
    try:
        created = versions.first().revision.date_created
        if created > datetime.datetime(2014, 11, 5):
            count = count + 1
            print(count, created, 'count:', len(versions), 'mpa_id:', m.mpa_id, 'wdpa_id:', m.wdpa_id)
    except:
        pass

1 2014-11-05 16:34:25.269322 count: 1 mpa_id: 7472 wdpa_id: 555512098
2 2018-10-23 16:20:11.786294 count: 8 mpa_id: 68808360 wdpa_id: 109032
3 2014-11-05 16:34:16.231142 count: 1 mpa_id: 6922 wdpa_id: 220021
4 2014-11-05 16:23:41.865516 count: 1 mpa_id: 12852 wdpa_id: 555526942
5 2014-11-05 16:23:40.497609 count: 1 mpa_id: 12830 wdpa_id: 555526788
6 2014-11-05 16:34:16.205659 count: 1 mpa_id: 6921 wdpa_id: 220012
7 2014-11-05 16:23:42.604099 count: 1 mpa_id: 12856 wdpa_id: 555526948
8 2014-11-05 16:23:37.325001 count: 1 mpa_id: 12785 wdpa_id: 555526153
9 2014-11-05 16:34:25.140890 count: 1 mpa_id: 7467 wdpa_id: 555512086
10 2014-11-05 16:21:09.253238 count: 1 mpa_id: 9525 wdpa_id: 555547619
11 2014-11-05 16:21:09.143455 count: 1 mpa_id: 9519 wdpa_id: 555547613
12 2014-11-05 16:21:09.236046 count: 1 mpa_id: 9524 wdpa_id: 555547618
13 2014-11-05 16:26:03.150065 count: 1 mpa_id: 14554 wdpa_id: 555547980
14 2014-11-05 16:22:32.442634 count: 1 mpa_id: 11560 wdpa_id: 346000
15 2014-11-05 16:

151 2014-11-05 16:22:32.380030 count: 1 mpa_id: 11559 wdpa_id: 345977
152 2014-11-05 16:22:32.308833 count: 1 mpa_id: 11557 wdpa_id: 345902
153 2015-06-08 09:03:18.491159 count: 3 mpa_id: 6357 wdpa_id: 15754
154 2014-11-05 16:22:09.625261 count: 1 mpa_id: 11077 wdpa_id: 193350
155 2014-11-05 16:23:33.461431 count: 1 mpa_id: 12734 wdpa_id: 555525804
156 2014-11-05 16:25:56.832736 count: 1 mpa_id: 14382 wdpa_id: 555547501
157 2014-11-05 16:23:38.050314 count: 1 mpa_id: 12797 wdpa_id: 555526177
158 2016-04-26 15:07:33.556004 count: 2 mpa_id: 7701010 wdpa_id: 12878
159 2014-11-05 16:21:08.228534 count: 1 mpa_id: 9471 wdpa_id: 109023
160 2014-11-05 16:23:43.161981 count: 1 mpa_id: 12858 wdpa_id: 555526950
161 2014-11-05 16:23:41.181804 count: 1 mpa_id: 12839 wdpa_id: 555526868
162 2014-11-05 16:23:41.226615 count: 1 mpa_id: 12840 wdpa_id: 555526871
163 2014-11-05 16:23:41.668499 count: 1 mpa_id: 12849 wdpa_id: 555526903
164 2014-11-05 16:23:40.879164 count: 1 mpa_id: 12837 wdpa_id: 55552685

267 2016-04-26 13:41:13.767213 count: 2 mpa_id: 7700730 wdpa_id: 9141
268 2015-11-18 15:33:18.940337 count: 3 mpa_id: 2919 wdpa_id: 147279
269 2014-11-05 16:23:37.416734 count: 1 mpa_id: 12786 wdpa_id: 555526154
270 2016-12-27 10:47:37.244795 count: 3 mpa_id: 560 wdpa_id: 6317
271 2014-11-05 16:24:59.684729 count: 1 mpa_id: 13729 wdpa_id: 555539444
272 2015-05-14 13:19:13.749573 count: 3 mpa_id: 1225 wdpa_id: 15153
273 2014-11-05 16:23:42.200182 count: 1 mpa_id: 12854 wdpa_id: 555526946
274 2015-05-26 12:59:20.636384 count: 1 mpa_id: 7700093 wdpa_id: 663
275 2014-11-05 16:25:29.730461 count: 1 mpa_id: 14048 wdpa_id: 555543095
276 2014-11-05 16:24:57.113620 count: 1 mpa_id: 13697 wdpa_id: 555539373
277 2014-11-05 16:24:58.478119 count: 1 mpa_id: 13716 wdpa_id: 555539393
278 2015-05-14 13:22:39.386669 count: 4 mpa_id: 583 wdpa_id: 7165
279 2014-11-05 16:24:59.733878 count: 1 mpa_id: 13730 wdpa_id: 555539445
280 2014-11-05 16:23:35.160126 count: 1 mpa_id: 12760 wdpa_id: 555526107
281 2014

385 2014-11-05 16:23:42.440567 count: 1 mpa_id: 12855 wdpa_id: 555526947
386 2014-11-05 16:23:39.870300 count: 1 mpa_id: 12816 wdpa_id: 555526294
387 2014-11-05 16:38:18.950815 count: 1 mpa_id: 2704 wdpa_id: 103548
388 2014-11-05 16:23:37.555483 count: 1 mpa_id: 12789 wdpa_id: 555526163
389 2014-11-05 16:24:56.969076 count: 1 mpa_id: 13692 wdpa_id: 555539368
390 2015-06-02 12:27:52.523150 count: 4 mpa_id: 2536 wdpa_id: 95335
391 2014-11-05 16:24:55.738927 count: 1 mpa_id: 13673 wdpa_id: 555539255
392 2014-11-05 16:21:30.596864 count: 1 mpa_id: 9855 wdpa_id: 106767
393 2015-06-02 12:31:19.590245 count: 1 mpa_id: 7702535 wdpa_id: 95331
394 2014-11-05 16:24:59.601360 count: 1 mpa_id: 13727 wdpa_id: 555539442
395 2014-11-05 16:24:58.578506 count: 1 mpa_id: 13717 wdpa_id: 555539394
396 2014-11-05 16:24:56.998096 count: 1 mpa_id: 13693 wdpa_id: 555539369
397 2014-11-05 16:24:57.024723 count: 1 mpa_id: 13694 wdpa_id: 555539370


In [4]:
from importlib import reload
from wdpa import merge
reload(merge)

<module 'wdpa.merge' from '/home/mpatlas/projects/django_mpatlas/wdpa/merge.py'>

In [73]:
remove_log = '/home/mpatlas/workspace/log/removewdpa_FRA_log.json'
wdpa2remove = merge.getRemoveWdpaList(logfile=remove_log)

In [74]:
len(wdpa2remove)

22

In [75]:
rm_mpas = Mpa.objects.filter(wdpa_id__in=wdpa2remove).order_by('country')
len(rm_mpas)

22

In [76]:
for m in rm_mpas:
    print(m.mpa_id, m.wdpa_id, m.country, m.sovereign, m.is_mpa, m.no_take, m.no_take_area, m.name, m.designation_eng)

8807050 306208 FRA FRA False Not Reported None Calvi Fishery Reserve
8801253 15998 FRA FRA False Not Reported None La Ribère Land acquired by Littoral and Lakeside Conservatory
8807066 306656 FRA FRA True Part 2.95 Cote Bleue Marine Park, Marine Protected Area
8801258 16041 FRA FRA False Not Reported None Le Mazet Land acquired by Littoral and Lakeside Conservatory
8801259 16045 FRA FRA True Not Reported None La Fontasse Land acquired by Littoral and Lakeside Conservatory
8801262 16055 FRA FRA True Not Reported None Presqu'île de Giens Land acquired by Littoral and Lakeside Conservatory
8807043 306177 FRA FRA True Part 12.0 Bouches de Bonifacio Nature Reserve
8807069 308527 FRA FRA False Not Reported None Carry-le-Rouet Marine Protected Zone: aquaculture concession
8807209 342574 FRA FRA False Not Reported None Îles Glorieuses Nature Reserve
8807211 342576 FRA FRA True Not Reported None Ilot de Bassas da India Nature Reserve
8807236 342624 FRA FRA True Not Reported None Juan de Nova Na

In [77]:
merge.removeMpasByWdpaId(wdpa2remove)

Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Ac

In [78]:
reload(merge)

<module 'wdpa.merge' from '/home/mpatlas/projects/django_mpatlas/wdpa/merge.py'>

In [83]:
wdpa2add = merge.getAddWdpaList()

In [84]:
len(wdpa2add)

136

In [85]:
wdpapid2add = merge.getAddWdpaPidList(verbose=True)

345888_D : 1 processed
345888_E : 2 processed
345888_F : 3 processed


In [87]:
len(wdpapid2add)

3

In [16]:
# Assign records with zones to be the first zone.  Do not rerun.
# In separate notebook

In [88]:
allpid2add = wdpapid2add + [str(int(i)) for i in wdpa2add]
len(allpid2add)

139

In [89]:
reload(merge)
addpoint_geolog = '/home/mpatlas/workspace/log/addpoint_FRA_log.json'
addpoly_log = '/home/mpatlas/workspace/log/addpoly_FRA_log.json'
addpoly_geolog = '/home/mpatlas/workspace/log/addpoly_FRA_geo_log.json'

In [90]:
points = Wdpa2018Point.objects.filter(wdpa_pid__in=allpid2add).order_by('wdpa_pid')
merge.updateMpasFromWdpaQueryset(qs=points, poly=False, logfile=None, geologfile=addpoint_geolog, dryrun=True)

True

In [91]:
# post_save.disconnect(mpa_post_save, sender=Mpa)
polys = Wdpa2018Poly.objects.filter(wdpa_pid__in=allpid2add).order_by('wdpa_pid')
merge.updateMpasFromWdpaQueryset(qs=polys, poly=True, logfile=addpoly_log, geologfile=addpoly_geolog, dryrun=True)
# post_save.connect(mpa_post_save, sender=Mpa)

1/138 adding/updating wdpa_pid 106713 with mpa_id None
Carto Error for mpa_id 68809049: Access denied
2/138 adding/updating wdpa_pid 106722 with mpa_id 2706
    PID: 106722 has 8 field diffs  and GEOM diff
Carto Error for mpa_id 2706: Access denied
3/138 adding/updating wdpa_pid 13407 with mpa_id 1135
    PID: 13407 has 10 field diffs  and GEOM diff
Carto Error for mpa_id 1135: Access denied
4/138 adding/updating wdpa_pid 13409 with mpa_id None
Carto Error for mpa_id 68809050: Access denied
5/138 adding/updating wdpa_pid 13413 with mpa_id 1139
    PID: 13413 has 9 field diffs  and GEOM diff
Carto Error for mpa_id 1139: Access denied
6/138 adding/updating wdpa_pid 147294 with mpa_id 2920
    PID: 147294 has 9 field diffs  and GEOM diff
Carto Error for mpa_id 2920: Access denied
7/138 adding/updating wdpa_pid 147302 with mpa_id 2925
    PID: 147302 has 9 field diffs  and GEOM diff
Carto Error for mpa_id 2925: Access denied
8/138 adding/updating wdpa_pid 147314 with mpa_id 2930
    PID: 1

Carto Error for mpa_id 68809105: Access denied
73/138 adding/updating wdpa_pid 555539599 with mpa_id None
Carto Error for mpa_id 68809106: Access denied
74/138 adding/updating wdpa_pid 555544124 with mpa_id None
Carto Error for mpa_id 68809107: Access denied
75/138 adding/updating wdpa_pid 555544125 with mpa_id None
Carto Error for mpa_id 68809108: Access denied
76/138 adding/updating wdpa_pid 555544126 with mpa_id None
Carto Error for mpa_id 68809109: Access denied
77/138 adding/updating wdpa_pid 555544127 with mpa_id None
Carto Error for mpa_id 68809110: Access denied
78/138 adding/updating wdpa_pid 555544128 with mpa_id None
Carto Error for mpa_id 68809111: Access denied
79/138 adding/updating wdpa_pid 555544129 with mpa_id None
Carto Error for mpa_id 68809112: Access denied
80/138 adding/updating wdpa_pid 555544130 with mpa_id None
Carto Error for mpa_id 68809113: Access denied
81/138 adding/updating wdpa_pid 555544131 with mpa_id None
Carto Error for mpa_id 68809114: Access denied

True

In [92]:
reload(merge)
updatepoint_geolog = '/home/mpatlas/workspace/log/updatepoint_FRA_log.json'
updatepoly_log = '/home/mpatlas/workspace/log/updatepoly_FRA_log.json'
updatepoly_geolog = '/home/mpatlas/workspace/log/updatepoly_FRA_geo_log.json'

In [93]:
wdpa_filter = (
    Q(iso3__icontains='FRA') | Q(parent_iso3__icontains='FRA')
    | Q(iso3__icontains='ATF') | Q(iso3__icontains='BLM') | Q(iso3__icontains='GLP')
    | Q(iso3__icontains='GUF') | Q(iso3__icontains='MAF') | Q(iso3__icontains='MTQ')
    | Q(iso3__icontains='MYT') | Q(iso3__icontains='NCL') | Q(iso3__icontains='PYF')
    | Q(iso3__icontains='REU') | Q(iso3__icontains='SHN') | Q(iso3__icontains='SYC')
)
polys = Wdpa2018Poly.objects.exclude(wdpa_pid__in=allpid2add).filter(wdpa_filter).filter(marine__in=('1','2')).order_by('wdpa_pid').values_list('wdpa_pid', flat=True)
points = Wdpa2018Point.objects.exclude(wdpa_pid__in=allpid2add).filter(wdpa_filter).filter(marine__in=('1','2')).order_by('wdpa_pid').values_list('wdpa_pid', flat=True)
allpid2update = list(set(list(polys) + list(points)))



In [94]:
len(allpid2update)

282

In [95]:
updatepoints = Wdpa2018Point.objects.filter(wdpa_pid__in=allpid2update)
merge.updateMpasFromWdpaQueryset(qs=updatepoints, poly=False, logfile=None, geologfile=updatepoint_geolog, dryrun=True)

1/42 adding/updating wdpa_pid 478034 with mpa_id 7262
    PID: 478034 has 6 field diffs  and GEOM diff
ERROR processing mpa 7262:  'NoneType' object has no attribute 'hexewkb'
2/42 adding/updating wdpa_pid 478035 with mpa_id 7263
    PID: 478035 has 6 field diffs  and GEOM diff
ERROR processing mpa 7263:  'NoneType' object has no attribute 'hexewkb'
3/42 adding/updating wdpa_pid 555542730 with mpa_id 7707519
    PID: 555542730 has 8 field diffs  and GEOM diff
ERROR processing mpa 7707519:  'NoneType' object has no attribute 'hexewkb'
4/42 adding/updating wdpa_pid 6940 with mpa_id 7705983
    PID: 6940 has 6 field diffs  and GEOM diff
ERROR processing mpa 7705983:  'NoneType' object has no attribute 'hexewkb'
5/42 adding/updating wdpa_pid 62224 with mpa_id 7706669
    PID: 62224 has 6 field diffs  and GEOM diff
ERROR processing mpa 7706669:  'NoneType' object has no attribute 'hexewkb'
6/42 adding/updating wdpa_pid 62226 with mpa_id 7706670
    PID: 62226 has 6 field diffs  and GEOM dif

True

In [96]:
updatepolys = Wdpa2018Poly.objects.filter(wdpa_pid__in=allpid2update)
merge.updateMpasFromWdpaQueryset(qs=updatepolys, poly=True, logfile=updatepoly_log, geologfile=updatepoly_geolog, dryrun=True)

1/266 adding/updating wdpa_pid 555559749 with mpa_id 15124
    PID: 555559749 has 8 field diffs  and GEOM diff
Carto Error for mpa_id 15124: Access denied
2/266 adding/updating wdpa_pid 147435 with mpa_id 9906
    PID: 147435 has 4 field diffs  and GEOM diff
Carto Error for mpa_id 9906: Access denied
3/266 adding/updating wdpa_pid 345972 with mpa_id 11558
    PID: 345972 has 8 field diffs  and GEOM diff
Carto Error for mpa_id 11558: Access denied
4/266 adding/updating wdpa_pid 555547508 with mpa_id 14388
    PID: 555547508 has 7 field diffs  and GEOM diff
Carto Error for mpa_id 14388: Access denied
5/266 adding/updating wdpa_pid 127842 with mpa_id 6831
    PID: 127842 has 6 field diffs 
Carto Error for mpa_id 6831: Access denied
6/266 adding/updating wdpa_pid 127842 with mpa_id 7706831
    PID: 127842 has 8 field diffs 
Carto Error for mpa_id 7706831: Access denied
7/266 adding/updating wdpa_pid 902857 with mpa_id 7707436
    PID: 902857 has 9 field diffs 
Carto Error for mpa_id 770743

56/266 adding/updating wdpa_pid 555559581 with mpa_id 15121
    PID: 555559581 has 6 field diffs  and GEOM diff
Carto Error for mpa_id 15121: Access denied
57/266 adding/updating wdpa_pid 178244 with mpa_id 10861
    PID: 178244 has 5 field diffs  and GEOM diff
Carto Error for mpa_id 10861: Access denied
58/266 adding/updating wdpa_pid 345900 with mpa_id 11556
    PID: 345900 has 4 field diffs  and GEOM diff
Carto Error for mpa_id 11556: Access denied
59/266 adding/updating wdpa_pid 345902 with mpa_id 11557
    PID: 345902 has 4 field diffs  and GEOM diff
Carto Error for mpa_id 11557: Access denied
60/266 adding/updating wdpa_pid 147483 with mpa_id 9908
    PID: 147483 has 5 field diffs  and GEOM diff
Carto Error for mpa_id 9908: Access denied
61/266 adding/updating wdpa_pid 83146 with mpa_id 9831
    PID: 83146 has 6 field diffs  and GEOM diff
Carto Error for mpa_id 9831: Access denied
62/266 adding/updating wdpa_pid 193356 with mpa_id 11078
    PID: 193356 has 4 field diffs  and GEOM

Carto Error for mpa_id 13698: Access denied
112/266 adding/updating wdpa_pid 555539381 with mpa_id 13705
    PID: 555539381 has 5 field diffs  and GEOM diff
Carto Error for mpa_id 13705: Access denied
113/266 adding/updating wdpa_pid 555539390 with mpa_id 13713
    PID: 555539390 has 5 field diffs  and GEOM diff
Carto Error for mpa_id 13713: Access denied
114/266 adding/updating wdpa_pid 555539392 with mpa_id 13715
    PID: 555539392 has 6 field diffs  and GEOM diff
Carto Error for mpa_id 13715: Access denied
115/266 adding/updating wdpa_pid 555539382 with mpa_id 13706
    PID: 555539382 has 5 field diffs  and GEOM diff
Carto Error for mpa_id 13706: Access denied
116/266 adding/updating wdpa_pid 555539372 with mpa_id 13696
    PID: 555539372 has 5 field diffs  and GEOM diff
Carto Error for mpa_id 13696: Access denied
117/266 adding/updating wdpa_pid 555539391 with mpa_id 13714
    PID: 555539391 has 5 field diffs  and GEOM diff
Carto Error for mpa_id 13714: Access denied
118/266 adding

164/266 adding/updating wdpa_pid 555526143 with mpa_id 12778
    PID: 555526143 has 3 field diffs  and GEOM diff
Carto Error for mpa_id 12778: Access denied
165/266 adding/updating wdpa_pid 555526144 with mpa_id 12779
    PID: 555526144 has 3 field diffs  and GEOM diff
Carto Error for mpa_id 12779: Access denied
166/266 adding/updating wdpa_pid 555525743 with mpa_id 12732
    PID: 555525743 has 7 field diffs  and GEOM diff
Carto Error for mpa_id 12732: Access denied
167/266 adding/updating wdpa_pid 555525807 with mpa_id 12737
    PID: 555525807 has 3 field diffs  and GEOM diff
Carto Error for mpa_id 12737: Access denied
168/266 adding/updating wdpa_pid 555525809 with mpa_id 12739
    PID: 555525809 has 3 field diffs  and GEOM diff
Carto Error for mpa_id 12739: Access denied
169/266 adding/updating wdpa_pid 555525932 with mpa_id 5769
    PID: 555525932 has 3 field diffs  and GEOM diff
Carto Error for mpa_id 5769: Access denied
170/266 adding/updating wdpa_pid 555525932 with mpa_id 77057

    PID: 555526890 has 2 field diffs  and GEOM diff
Carto Error for mpa_id 5772: Access denied
217/266 adding/updating wdpa_pid 555526890 with mpa_id 7705772
    PID: 555526890 has 12 field diffs  and GEOM diff
Carto Error for mpa_id 7705772: Access denied
218/266 adding/updating wdpa_pid 555526747 with mpa_id 12826
    PID: 555526747 has 4 field diffs  and GEOM diff
Carto Error for mpa_id 12826: Access denied
219/266 adding/updating wdpa_pid 555526858 with mpa_id 12836
    PID: 555526858 has 4 field diffs  and GEOM diff
Carto Error for mpa_id 12836: Access denied
220/266 adding/updating wdpa_pid 555526859 with mpa_id 12837
    PID: 555526859 has 4 field diffs  and GEOM diff
Carto Error for mpa_id 12837: Access denied
221/266 adding/updating wdpa_pid 555526082 with mpa_id 12756
    PID: 555526082 has 4 field diffs  and GEOM diff
Carto Error for mpa_id 12756: Access denied
222/266 adding/updating wdpa_pid 555526152 with mpa_id 12784
    PID: 555526152 has 4 field diffs  and GEOM diff
Ca

True

In [99]:
mpaset.filter(wdpa_pid__isnull=False).update(
    no_take_wdpa=F('no_take'), no_take_area_wdpa=F('no_take_area')
)

583

In [100]:
nt = ('All', 'Part', 'None')
ntupdate = mpaset.filter(wdpa_pid__isnull=False).filter(
    (~Q(no_take_wdpa__in=('All')) & Q(no_take_mpatlas__in=('All')))
    | (~Q(no_take_wdpa__in=('All', 'Part')) & Q(no_take_mpatlas__in=('All', 'Part')))
    | (Q(no_take_wdpa__in=('Not Applicable')) & Q(no_take_mpatlas__in=('All', 'Part', 'None')))
)
ntupdate.update(no_take=F('no_take_mpatlas'))

8

In [101]:
ntareaupdate = mpaset.filter(wdpa_pid__isnull=False).filter(
    Q(no_take_area_mpatlas__isnull=False) &
    (
        Q(no_take_area_wdpa__isnull=True)
        | Q(no_take_area_mpatlas__gt=F('no_take_area_wdpa'))
    )
)
ntareaupdate.update(no_take_area=F('no_take_area_mpatlas'))

8

In [113]:
ntchanges = ntupdate | ntareaupdate
for n in ntchanges:
    print(n.mpa_id, n.wdpa_id, n.wdpa_pid, n.name, n.designation, n.designation_eng, n.no_take_wdpa, n.no_take_mpatlas, n.no_take_area_wdpa, n.no_take_area_mpatlas)

68808359 345888 345888_D Réserve Naturelle Nationale des Terres australes françaises Réserve Naturelle Nationale National Nature Reserve All All 120234.0 120455.375836
7700584 7168 7168 Scandola Réserve naturelle de Corse Corsican Nature Reserve Not Reported Part 0.0 None
9904 147297 147297 Guadeloupe Parc national, zone cœur National Park - Core Area Not Reported All 0.0 0.0
7468 555512087 555512087 Atolls d'Entrecasteaux Aire Marine Protégée Marine Protected Area Not Reported All 0.0 1068.0
7700730 9141 9141 Baie Ternay Marine National Park Marine National Park All All 0.0 0.8628
414 4044 4044 Cerbère - Banyuls Réserve naturelle nationale National Nature Reserve Not Reported Part 0.0 0.65
93 663 663 Port-Cros Parc national, zone cœur National Park - Core Area Not Reported Part 0.0 0.65
584 7168 7168 Scandola Réserve naturelle de Corse Corsican Nature Reserve Not Reported Part 0.0 0.72
7705197 365016 365016 Port-Cros Specially Protected Areas of Mediterranean Importance (Barcelona Con

In [103]:
ntchanges = ntupdate | ntareaupdate

import csv

with open('/home/mpatlas/workspace/log/notakechange_FRA_log.csv', 'w', newline='') as csvfile:
    fieldnames = ['mpa_id', 'wdpa_id', 'wdpa_pid', 'name', 'designation', 'designation_eng', 'no_take_wdpa', 'no_take_mpatlas', 'no_take_area_wdpa', 'no_take_area_mpatlas']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in ntchanges.values(*fieldnames):
        writer.writerow(row)

In [115]:
for n in ntchanges:
    try:
        w2018 = Wdpa2018Poly.objects.get(wdpa_pid=n.wdpa_pid)
    except:
        w2018 = Wdpa2018Point.objects.get(wdpa_pid=n.wdpa_pid)
    try:
        w2014 = Wdpa2014Polygon.objects.filter(wdpa_pid=F('wdpa_pid'))[0]
    except:
        try:
            w2014 = Wdpa2014Point.objects.filter(wdpa_pid=F('wdpa_pid'))[0]
        except:
            try:
                w2014 = Wdpa2014Polygon.objects.filter(wdpa_id=F('wdpa_id'))[0]
            except:
                w2014 = Wdpa2014Point.objects.filter(wdpa_id=F('wdpa_id'))[0]
    if (n.no_take == w2014.no_take) or (n.no_take_area == w2014.no_tk_area):
        print(w2018.name, w2018.desig, n.no_take_mpatlas, w2018.no_take, w2014.no_take, n.no_take_area_mpatlas, w2018.no_tk_area, w2014.no_tk_area)

Scandola Réserve naturelle de Corse Part Not Reported Not Applicable None 0.0 0.0
Guadeloupe Parc national, zone cœur All Not Reported Not Applicable 0.0 0.0 0.0
Port-Cros Specially Protected Areas of Mediterranean Importance (Barcelona Convention) Part Not Reported Not Applicable None 0.0 0.0


In [116]:
w2014 = Wdpa2018Point.objects.get(wdpaid=555512087)
w2014.no_take

'Not Reported'

In [112]:
ntchanges2 = ntupdate | ntareaupdate
ntchanges.count()

11

In [9]:
mpaset = mpas_all_nogeom.filter(
    Q(country__icontains='FRA') | Q(sovereign__icontains='FRA')
    | Q(country__icontains='ATF') | Q(country__icontains='BLM') | Q(country__icontains='GLP')
    | Q(country__icontains='GUF') | Q(country__icontains='MAF') | Q(country__icontains='MTQ')
    | Q(country__icontains='MYT') | Q(country__icontains='NCL') | Q(country__icontains='PYF')
    | Q(country__icontains='REU') | Q(country__icontains='SHN') | Q(country__icontains='SYC')
)

from mpatlas.utils import cartompa
cartompa.purgeCartoMpas()
cartompa.updateAllMpas(mpas=mpaset)

Processing 10 of 622 mpa records at a time
Records [0 - 9]
Records [10 - 19]
ERROR processing mpa 9372:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 9372
ERROR processing mpa 9366:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 9366
ERROR processing mpa 9364:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 9364
ERROR processing mpa 9138:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 9138
ERROR processing mpa 68808143:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 68808143
ERROR processing mpa 9383:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 9383
ERROR processing mpa 9380:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 9380
ERROR processing mpa 60009529:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 60009529
Records [20 - 29]
ERROR processing mpa 68808192:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 68808192
ERROR processing mpa 9099:  'NoneType' object has no attribute

[9372,
 9366,
 9364,
 9138,
 68808143,
 9383,
 9380,
 60009529,
 68808192,
 9099,
 9384,
 9065,
 9195,
 9382,
 9381,
 9365,
 9370,
 9369,
 9104,
 9371,
 9385,
 60009492,
 60009536,
 60009491,
 9194,
 9368,
 9098,
 7706671,
 6692,
 7472,
 7470,
 7706747,
 7471,
 6747,
 7705984,
 7705944,
 5858,
 9462,
 6925,
 7468,
 7707519,
 7263,
 7705983,
 7706669,
 7706670,
 7705858,
 5943,
 7705943,
 5944,
 7706928,
 6928,
 9643,
 9526,
 7076,
 6662,
 7465,
 5927,
 9518,
 9520,
 7159,
 9521,
 9522,
 9525,
 9519,
 9524,
 7706357,
 7262,
 9523,
 6357]